In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras import models
from tensorflow.keras import layers
from sklearn.metrics import mean_squared_error

In [ ]:
DSMC = pd.read_csv('./Data/South Korea - Dongkuk Steel Mill Co Ltd (001230).csv')